# Работа с Pandas

Булыгин Олег  

* [LinkedIn](linkedin.com/in/obulygin)  
* [Telegram](https://t.me/obulygin91)  
* [facebook](fb.com/obulygin91)  
* [Vk](vk.com/obulygin91)  
* email: obulygin91@ya.ru  

[Сообщество по Python](https://yandex.ru/q/loves/pythontalk/) на Кью  
[Сообщество по Data Science и анализу данных](https://yandex.ru/q/loves/datatalk/) на Кью 

## План
В рамках воркфлоу работы с открытым датафреймом по реальным транзакциям британского онлайн-магазина подарков. Реализуем набор аналитических действий и решим задачи:
- посчитаем статистику по странам;
- проанализируем выручку;
- изучим статистику по клиентам;
- проанализируем динамику среднего чека;
- просегментируем товары по стоимости;
- сделаем когортный анализ и анализ удержания клиентов (уже непросто!).

Пройдя по этим шагам повторим:
- базовую работу с датами;
- преобразование типов;
- сортировку;
- группировку и сводные таблицы;
- объединение датафреймов.

In [ ]:
# импортируем pandas
import pandas as pd

### Открытие и знакомство с данными

In [ ]:
# читаем данные из csv, который находится в zip-файле на github
url = 'https://github.com/obulygin/SkillFactory/blob/main/ecommerce-data.zip?raw=true'
df = pd.read_csv(url, encoding='ISO-8859-1', compression='zip')

In [ ]:
# смотрим первые 5 строк таблицы
display(df.head())
# смотрим последние 5 строк таблицы
display(df.tail())

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


In [ ]:
# рассчитываем базовые статистики по каждому числовому признаку
df.describe() 

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


### Избавимся от отрицательных значений

In [ ]:
# оставляем данные только с положительными ценами на товар и количеством
df = df[(df['UnitPrice'] > 0) | (df['Quantity'] > 0)]
df.describe()

,Quantity,UnitPrice,CustomerID
count,540573.000000,540573.000000,406829.000000
mean,9.958705,4.622510,15287.690570
std,216.229554,96.879076,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


### Избавимся от дубликатов в данных

In [ ]:
# посмотрим, сколько у нас повторов
print(df.duplicated().sum())

5268


In [ ]:
# сколько всего строк?
print(df.shape)

(540573, 8)


In [ ]:
# удалим дубли и проверим
df.drop_duplicates(inplace=True)
print(df.shape)

(535305, 8)


### Поправим типы данных

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 535305 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    535305 non-null  object 
 1   StockCode    535305 non-null  object 
 2   Description  534713 non-null  object 
 3   Quantity     535305 non-null  int64  
 4   InvoiceDate  535305 non-null  object 
 5   UnitPrice    535305 non-null  float64
 6   CustomerID   401604 non-null  float64
 7   Country      535305 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.8+ MB


In [ ]:
# почему ошибка?
df['CustomerID'] = df['CustomerID'].astype('int64')

IntCastingNaNError: ignored

In [ ]:
# Nullable datatypes: https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html
df['CustomerID'] = df['CustomerID'].astype('Int64')

In [ ]:
# переведем даты из строкового формата в datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 535305 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    535305 non-null  object        
 1   StockCode    535305 non-null  object        
 2   Description  534713 non-null  object        
 3   Quantity     535305 non-null  int64         
 4   InvoiceDate  535305 non-null  datetime64[ns]
 5   UnitPrice    535305 non-null  float64       
 6   CustomerID   401604 non-null  Int64         
 7   Country      535305 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 37.3+ MB


### Изучаем статистику по странам

In [ ]:
# считаем сколько транзакций было по странам при помощи value_counts
df['Country'].value_counts()

United Kingdom          488964
Germany                   9480
France                    8541
EIRE                      8184
Spain                     2528
Netherlands               2371
Belgium                   2069
Switzerland               1994
Portugal                  1510
Australia                 1258
Norway                    1086
Italy                      803
Channel Islands            757
Finland                    695
Cyprus                     611
Sweden                     461
Unspecified                442
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     294
USA                        291
Hong Kong                  284
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [ ]:
# считаем сколько уникальных клиентов в странах
countries = df.groupby('Country')['CustomerID'].nunique().sort_values(ascending=False)
countries

Country
United Kingdom          3950
Germany                   95
France                    87
Spain                     31
Belgium                   25
Switzerland               21
Portugal                  19
Italy                     15
Finland                   12
Austria                   11
Norway                    10
Netherlands                9
Australia                  9
Channel Islands            9
Denmark                    9
Cyprus                     8
Sweden                     8
Japan                      8
Poland                     6
USA                        4
Canada                     4
Unspecified                4
Israel                     4
Greece                     4
EIRE                       3
Malta                      2
United Arab Emirates       2
Bahrain                    2
Lithuania                  1
Czech Republic             1
Lebanon                    1
RSA                        1
Saudi Arabia               1
Singapore                  1
Icelan

#### Практика. Определяем топ-5 лояльных клиентов по количеству покупок

### Создадим временные признакми

In [ ]:
# добавим месяц покупки в новый столбец при помощи dt.strftime('%Y - %m')
df['InvoiceMonth'] = df['InvoiceDate'].dt.strftime('%Y-%m') 

In [ ]:
# уникальные покупатели по месяцам
df.groupby('InvoiceMonth')['CustomerID'].nunique()

InvoiceMonth
2010-12     948
2011-01     783
2011-02     798
2011-03    1020
2011-04     899
2011-05    1079
2011-06    1051
2011-07     993
2011-08     980
2011-09    1302
2011-10    1425
2011-11    1711
2011-12     686
Name: CustomerID, dtype: int64

In [ ]:
# зафиксируем дни недели
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek + 1

In [ ]:
# посчитаем количество заказов по дням недели
df.groupby('DayOfWeek')['InvoiceNo'].nunique()


DayOfWeek
1    3955
2    4466
3    4538
4    5379
5    3845
7    2381
Name: InvoiceNo, dtype: int64

#### Практика. Изучим динамику количества покупок по часам

### Посчитаем выручку

In [ ]:
# создадим новый столбец
df['Revenue'] = df['Quantity'] * df['UnitPrice']

df.head() 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,DayOfWeek,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12,3,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,3,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12,3,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,3,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,3,20.34


In [ ]:
# выручка по месяцам
df.groupby('InvoiceMonth')['Revenue'].sum()

InvoiceMonth
2010-12     746723.610
2011-01     558448.560
2011-02     497026.410
2011-03     682013.980
2011-04     492367.841
2011-05     722094.100
2011-06     689977.230
2011-07     680156.991
2011-08     681386.460
2011-09    1017596.682
2011-10    1069368.230
2011-11    1456145.800
2011-12     432701.060
Name: Revenue, dtype: float64

#### Практика. Найдите 3 страны, которые принесли максимальную выручку после UK

### Сегменты товаров по стоимости

In [ ]:
# напишем функцию для сегментации товаров
def price_segment(row):
    if row < 5:
        return 'cheap' 
    if 5 <= row < 25:
        return 'lower-middle'
    if 25 <= row < 100:
        return 'upper-middle'
    if row >= 100:
        return 'expensive'
 

df['PriceSegment'] = df['UnitPrice'].apply(price_segment)
df['PriceSegment'].value_counts()

cheap           447735
lower-middle     84939
upper-middle      1594
expensive         1037
Name: PriceSegment, dtype: int64

#### Практика. Определить, какая из категорий товаров принесла больше всего выручки

### Как меняется средний чек со временем?

In [ ]:
df.groupby('InvoiceMonth')['Revenue'].sum().divide(df.groupby('InvoiceMonth')['InvoiceNo'].nunique())

InvoiceMonth
2010-12    381.955811
2011-01    404.672870
2011-02    369.536364
2011-03    368.855587
2011-04    316.025572
2011-05    353.102249
2011-06    362.192772
2011-07    375.777343
2011-08    403.904244
2011-09    456.526102
2011-10    429.292746
2011-11    437.938586
2011-12    439.290416
dtype: float64

### Построим сводные таблицы

In [ ]:
# сводная таблица по выручки в определенной стране в разрезе пользователей и месяцев
country = 'France'
table = pd.pivot_table(
    df[df['Country']==country], 
    index='CustomerID', 
    columns='InvoiceMonth', 
    aggfunc=sum, 
    values='Revenue', 
    fill_value=0)


# подсветить максимальные значения по каждому месяцу
# table
table.style.highlight_max()
# table.style.background_gradient(cmap='PuBu')

InvoiceMonth,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12
CustomerID,,,,,,,,,,,,,
12413,0.000000,415.300000,0.000000,0.000000,84.650000,0.000000,0.000000,0.000000,0.000000,0.000000,194.200000,0.000000,0.000000
12437,0.000000,337.950000,590.260000,147.710000,209.100000,635.320000,501.640000,0.000000,0.000000,557.280000,263.460000,1488.040000,165.900000
12441,173.550000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12488,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,553.390000,214.060000,531.210000,0.000000
12489,0.000000,334.930000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12490,0.000000,864.220000,0.000000,0.000000,0.000000,331.500000,484.060000,412.930000,424.060000,1035.820000,0.000000,1403.040000,462.300000
12491,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,460.400000,0.000000
12493,0.000000,0.000000,0.000000,0.000000,0.000000,215.550000,201.240000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12494,48.750000,-10.950000,243.700000,0.000000,0.000000,403.090000,0.000000,135.320000,0.000000,0.000000,214.740000,104.700000,0.000000


#### Практика. Построить сводную таблицу c кол-вом заказов по странам и месяцам


In [ ]:
# pd.pivot_table(
#     df, 
#     index='Country', 
#     columns='InvoiceMonth', 
#     aggfunc='nunique', 
#     values='InvoiceNo', 
#     fill_value=0)

In [ ]:
# изучим сводную статистику по покупателям из произвольной страны (суммарная и средняя выручка)
products = df[df['Country']==country].groupby(['CustomerID', 'InvoiceNo'], as_index=False)['Revenue'].sum()
display(products)
products = products.groupby('CustomerID').agg({'InvoiceNo': ['count'], 'Revenue': ['sum','mean']})
display(products)

,CustomerID,InvoiceNo,Revenue
0,12413,540365,479.25
1,12413,550338,84.65
2,12413,569402,194.20
3,12413,C540367,-54.00
4,12413,C540853,-9.95
...,...,...,...
453,12740,570030,427.60
454,14277,552826,1629.03
455,14277,577152,499.40
456,14277,578541,2059.57


InvoiceNo  Revenue            
               count      sum        mean
CustomerID                               
12413              5   694.15  138.830000
12437             19  4896.66  257.718947
12441              1   173.55  173.550000
12488              3  1298.66  432.886667
12489              1   334.93  334.930000
...              ...      ...         ...
12734              1   110.40  110.400000
12735              2   779.57  389.785000
12736              1   234.00  234.000000
12740              1   427.60  427.600000
14277              4  3928.80  982.200000

[87 rows x 3 columns]

In [ ]:
# объединяем статистику по месяцам и статистику по выручки и количеству заказов в разрезе пользователей
table_plus = pd.merge(table, products, on='CustomerID') 
table_plus.style.highlight_max()
# table_plus.style.background_gradient(cmap='PuBu')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  


,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,"('InvoiceNo', 'count')","('Revenue', 'sum')","('Revenue', 'mean')"
CustomerID,,,,,,,,,,,,,,,,
12413,0.000000,415.300000,0.000000,0.000000,84.650000,0.000000,0.000000,0.000000,0.000000,0.000000,194.200000,0.000000,0.000000,5,694.150000,138.830000
12437,0.000000,337.950000,590.260000,147.710000,209.100000,635.320000,501.640000,0.000000,0.000000,557.280000,263.460000,1488.040000,165.900000,19,4896.660000,257.718947
12441,173.550000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,173.550000,173.550000
12488,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,553.390000,214.060000,531.210000,0.000000,3,1298.660000,432.886667
12489,0.000000,334.930000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,334.930000,334.930000
12490,0.000000,864.220000,0.000000,0.000000,0.000000,331.500000,484.060000,412.930000,424.060000,1035.820000,0.000000,1403.040000,462.300000,10,5417.930000,541.793000
12491,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,460.400000,0.000000,1,460.400000,460.400000
12493,0.000000,0.000000,0.000000,0.000000,0.000000,215.550000,201.240000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,416.790000,138.930000
12494,48.750000,-10.950000,243.700000,0.000000,0.000000,403.090000,0.000000,135.320000,0.000000,0.000000,214.740000,104.700000,0.000000,13,1139.350000,87.642308


### Когортный анализ

Идея когортного анализа состоит в том, чтобы резделить пользоватей на группы по определенным признакам, и отслеживать поведение этих групп во времени. Обычно группы пользователей (когорты) выделяют на основе недели (месяца), когда пользователи совершили первую покупку.

Выделив такие группы пользователей (когорты), вы следите за ними в течение времени и измеряете ключевые метрики для каждой отдельной когорты.

In [ ]:
# определяем месяц первой покупки для каждого покупателя 
cohorts = df.groupby('CustomerID', as_index=False)['InvoiceMonth'].min()
cohorts.rename(columns = {'InvoiceMonth':'Cohort'}, inplace=True)
cohorts.head()

,CustomerID,Cohort
0,12346,2011-01
1,12347,2010-12
2,12348,2010-12
3,12349,2011-11
4,12350,2011-02


In [ ]:
# добавим когорты к основным данным

# исключаем пропуски по пользователям для корректного объединения
df_merged = pd.merge(df.dropna(), cohorts, how='left', on='CustomerID')
df_merged.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,DayOfWeek,Revenue,PriceSegment,Cohort
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12,3,15.30,cheap,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,3,20.34,cheap,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12,3,22.00,cheap,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,3,20.34,cheap,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,3,20.34,cheap,2010-12


In [ ]:
# считаем доходы по когортам
df_merged.groupby('Cohort')['Revenue'].sum()

Cohort
2010-12    4401284.320
2011-01     998493.081
2011-02     548815.770
2011-03     592253.990
2011-04     320689.741
2011-05     270999.210
2011-06     222549.330
2011-07     134272.131
2011-08     183626.870
2011-09     226520.981
2011-10     204734.400
2011-11     147556.850
2011-12      26722.750
Name: Revenue, dtype: float64

### Анализ retention

Customer retention (удержание клиентов) — это способность компании удерживать своих потребителей в течение определенного периода времени.

In [ ]:
import numpy as np

# пишем функцию которая добавляет столбец с временем жизни каждой когорты
def cohort_period(df):
  df['CohortLT'] = np.arange(len(df))
  return df

# считаем сколько уникальных клиентов у нас в каждой когорте
cohorts = df_merged.groupby(['Cohort', 'InvoiceMonth'])[['CustomerID']].nunique()
cohorts = cohorts.groupby(level=0).apply(cohort_period)
cohorts.rename({'CustomerID': 'TotalUsers'}, inplace=True)
cohorts

CustomerID  CohortLT
Cohort  InvoiceMonth                      
2010-12 2010-12              948         0
        2011-01              362         1
        2011-02              317         2
        2011-03              367         3
        2011-04              341         4
...                          ...       ...
2011-10 2011-11               93         1
        2011-12               46         2
2011-11 2011-11              321         0
        2011-12               43         1
2011-12 2011-12               41         0

[91 rows x 2 columns]

In [ ]:
# переведем когорты в индекс, а дату чека в столбец
cohorts.reset_index(inplace=True)
cohorts.set_index(['Cohort', 'CohortLT'], inplace=True) 
cohorts = cohorts.rename(columns={'CustomerID': 'TotalUsers'})
cohorts

InvoiceMonth  TotalUsers
Cohort  CohortLT                         
2010-12 0             2010-12         948
        1             2011-01         362
        2             2011-02         317
        3             2011-03         367
        4             2011-04         341
...                       ...         ...
2011-10 1             2011-11          93
        2             2011-12          46
2011-11 0             2011-11         321
        1             2011-12          43
2011-12 0             2011-12          41

[91 rows x 2 columns]

In [ ]:
# считаем ретеншн пользователей в каждой когорте

# определим количество покупок в первый месяц когорты (100%)
cohort_group_size = cohorts['TotalUsers'].groupby(level=0).first()
cohort_group_size


Cohort
2010-12    948
2011-01    421
2011-02    380
2011-03    440
2011-04    299
2011-05    279
2011-06    235
2011-07    191
2011-08    167
2011-09    298
2011-10    352
2011-11    321
2011-12     41
Name: TotalUsers, dtype: int64

In [ ]:
# узнаем какой процент покупок этой когортой совершался в последующие месяца
user_retention = cohorts['TotalUsers'].unstack(0).divide(cohort_group_size, axis=1)
user_retention

Cohort,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12
CohortLT,,,,,,,,,,,,,
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
1,0.381857,0.239905,0.247368,0.190909,0.227425,0.236559,0.208511,0.209424,0.251497,0.298658,0.264205,0.133956,NaN
2,0.334388,0.282660,0.192105,0.254545,0.220736,0.172043,0.187234,0.204188,0.251497,0.325503,0.130682,NaN,NaN
3,0.387131,0.242280,0.278947,0.218182,0.210702,0.172043,0.272340,0.230366,0.251497,0.120805,NaN,NaN,NaN
4,0.359705,0.327791,0.268421,0.231818,0.207358,0.215054,0.246809,0.272251,0.137725,NaN,NaN,NaN,NaN
5,0.396624,0.299287,0.247368,0.177273,0.237458,0.243728,0.336170,0.115183,NaN,NaN,NaN,NaN,NaN
6,0.379747,0.261283,0.255263,0.263636,0.230769,0.265233,0.102128,NaN,NaN,NaN,NaN,NaN,NaN
7,0.354430,0.256532,0.281579,0.238636,0.260870,0.103943,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.354430,0.311164,0.257895,0.288636,0.083612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Спасибо за внимание буду рад ответить на ваши вопросы!**  
Ссылка на форму ОС:
https://forms.gle/y8xaFwJqtbFSjUeG8